In [1]:

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras import models
from keras import optimizers

import time
import threading
import optuna
from optuna_dashboard import wsgi
from wsgiref.simple_server import make_server


# Optuna Demo
Steven Glover <br>
cscie-89 Deep Learning <br>

# Data

https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images/

# Show size of data from each folder

In [2]:
# Define Directories
train_path = "D:/Analytics/Kaggle Data/lung_ct_scan/train/"
validation_path = "D:/Analytics/Kaggle Data/lung_ct_scan/valid/"
test_path = "D:/Analytics/Kaggle Data/lung_ct_scan/test/"

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.


# Simple model for baseline

In [9]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=10,
      validation_data=validation_generator,
      validation_steps=len(validation_generator))

Epoch 1/10
20/20 [==============================] - 25s 1s/step - loss: 4.1296 - acc: 0.3785 - val_loss: 1.2712 - val_acc: 0.4698
Epoch 2/10
20/20 [==============================] - 27s 1s/step - loss: 1.5397 - acc: 0.4747 - val_loss: 1.0294 - val_acc: 0.4508
Epoch 3/10
20/20 [==============================] - 24s 1s/step - loss: 1.2413 - acc: 0.5090 - val_loss: 1.1824 - val_acc: 0.5238
Epoch 4/10
20/20 [==============================] - 24s 1s/step - loss: 0.9737 - acc: 0.6297 - val_loss: 1.1457 - val_acc: 0.4508
Epoch 5/10
20/20 [==============================] - 25s 1s/step - loss: 0.7811 - acc: 0.7064 - val_loss: 1.0290 - val_acc: 0.4508
Epoch 6/10
20/20 [==============================] - 25s 1s/step - loss: 0.6815 - acc: 0.7162 - val_loss: 1.2214 - val_acc: 0.4159
Epoch 7/10
20/20 [==============================] - 24s 1s/step - loss: 0.5289 - acc: 0.7863 - val_loss: 1.2340 - val_acc: 0.5143
Epoch 8/10
20/20 [==============================] - 25s 1s/step - loss: 0.4858 - acc: 0.80

# Optuna
## 2.a Trial, Study, and Suggest APIs

The model began to drastically overfit after the 4th epoch. With this simple model architecture, the model began to overfit after the 4th epoch. Lets use Optuna to explore how we can improve the model. As a simple exmaple, to illustrate the API and its compenents, lets add some drop out to mitigate the overfitting. But how much dropout is optimal for this current architecture. 

### Dashboard and google colab
https://stackoverflow.com/questions/76033104/launching-optuna-dashboard-in-google-colaboratory

In [2]:
from google.colab import output

port = 1234
storage = optuna.storages.RDBStorage("sqlite:////content/drive/MyDrive/optimization.db")
app = wsgi(storage)
httpd = make_server("localhost", port, app)
thread = threading.Thread(target=httpd.serve_forever)
thread.start()
time.sleep(5)

output.serve_kernal_port_as_window(port, path="/dashboard/")